In [4]:
!pip install recordlinkage
!pip install fuzzymatcher

  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13905 sha256=ea5bc5413db56f5b1e89f29dd699aae926997f7bc6a8472eb49f14a91af42789
  Stored in directory: c:\users\bookw\appdata\local\pip\cache\wheels\1d\a8\cb\6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
Successfully built metaphone


You should consider upgrading via the 'c:\users\bookw\anaconda3\python.exe -m pip install --upgrade pip' command.


In [26]:
!pip install sqlite-fts4

You should consider upgrading via the 'c:\users\bookw\anaconda3\python.exe -m pip install --upgrade pip' command.


In [28]:
import pandas as pd
import recordlinkage
from pathlib import Path

import logging
logging.basicConfig(level=logging.DEBUG)

from pathlib import Path
import fuzzymatcher
import jellyfish

In [29]:
import sqlite3, platform

print('sqlite {}'.format(sqlite3.version_info))
print('sqlite {}'.format(sqlite3.sqlite_version_info))
print('sqlite {}'.format(sqlite3.__file__))

db = sqlite3.connect(':memory:') # memory stream
cursor = db.cursor()

results = cursor.execute("pragma compile_options")

for r in results:
    print(r)

print()
print("Operating System: {} {}".format(platform.system(), platform.release()))
print("Platform: {} {}".format(platform.python_implementation(),platform.python_version()))
print("SQLite: {}".format(sqlite3.sqlite_version))

sqlite (2, 6, 0)
sqlite (3, 30, 1)
sqlite C:\Users\bookw\Anaconda3\lib\sqlite3\__init__.py
('COMPILER=msvc-1916',)
('THREADSAFE=1',)

Operating System: Windows 10
Platform: CPython 3.7.3
SQLite: 3.30.1


In [30]:
#For Solving the FTS4 Problem
import sqlite3
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
conn.enable_load_extension(True)
for (val,) in cur.execute('pragma compile_options'): 
    print (val)

COMPILER=msvc-1916
THREADSAFE=1


In [14]:
sai_accounts = pd.read_csv('sai_organization.csv')
gots_accounts = pd.read_csv('gots_organization.csv')

In [15]:
sai_accounts.columns

Index(['address1', 'canonical_name1', 'city1', 'country1', 'country_code1',
       'id1', 'name1', 'source', 'state1', 'type', 'url', '_cached_page_id',
       'Id1'],
      dtype='object')

In [21]:
gots_accounts.columns

Index(['address2', 'canonical_name2', 'city2', 'country2', 'country_code2',
       'id', 'name2', 'source', 'state2', 'type', 'url', '_cached_page_id',
       'Id2'],
      dtype='object')

In [22]:
left_on = ['address1', 'canonical_name1', 'city1', 'country1', 'country_code1', 'name1', 'state1']

right_on = ['address2', 'canonical_name2', 'city2', 'country2', 'country_code2', 'name2', 'state2']

In [23]:
matched_results = fuzzymatcher.fuzzy_left_join(sai_accounts,
                                            gots_accounts,
                                            left_on,
                                            right_on,
                                            left_id_col = "Id1", right_id_col = "Id2")


OperationalError: no such module: fts4

In [19]:
'''
def get_closest_match(x, list_strings):
    best_match = None
    highest_jw = 0

    for current_string in list_strings:
        current_score = jellyfish.jaro_winkler(x, current_string)

        if(current_score > highest_jw):
            highest_jw = current_score
            best_match = current_string

    return best_match
'''

In [17]:
#organization_dupes = pd.read_csv('new_input_file_edited.csv', index_col='Id')

In [18]:
organization_dupes.columns

Index(['address', 'canonical_name', 'city', 'country', 'country_code', 'id',
       'name', 'source', 'state', 'type', 'url'],
      dtype='object')

In [19]:
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='country_code')
dupe_candidate_links = dupe_indexer.index(organization_dupes)

In [20]:
compare_dupes = recordlinkage.Compare()

compare_dupes.string('name',
                    'name',
                    threshold=0.80,
                    label='organization_name')
compare_dupes.string('address',
                    'address',
                    threshold=0.85,
                    label='organization_address')
compare_dupes.string('canonical_name',
                    'canonical_name',
                    threshold=0.85,
                    label='organization_canonical')
compare_dupes.exact('city', 'city', label='organization_city')

dupe_features = compare_dupes.compute(dupe_candidate_links, organization_dupes)

In [21]:
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

4.0         949
3.0         100
2.0       16639
1.0       99939
0.0    33419414
dtype: int64

In [22]:
potential_dupes = dupe_features[dupe_features.sum(axis=1) > 1].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'organization_name':'organization_city'].sum(axis=1)

In [23]:
potential_dupes

,Id_1,Id_2,organization_name,organization_address,organization_canonical,organization_city,Score
0,11906,10579,1.0,0.0,1.0,0,2.0
1,12427,10046,1.0,0.0,1.0,0,2.0
2,3,1,1.0,1.0,1.0,0,3.0
3,4,2,1.0,1.0,1.0,0,3.0
4,6,5,1.0,0.0,1.0,0,2.0
...,...,...,...,...,...,...,...
17683,14266,14220,1.0,0.0,1.0,0,2.0
17684,14266,14222,1.0,0.0,1.0,0,2.0
17685,14266,14228,1.0,0.0,1.0,0,2.0
17686,14266,14244,1.0,0.0,1.0,0,2.0


In [26]:
organization_dupes.loc[11906, :]

address                                            NaN
canonical_name                                 shaehan
city                                               NaN
country                                    South Korea
country_code                                        KR
id                fd3cd261-f835-4129-8d61-9b946895a0aa
name                                           SHAEHAN
source            9b50d4d5-6c83-4ae8-8c47-5f722e35a78b
state                                              NaN
type              905e0790-81a3-4396-acb6-e715ce483f6a
url                                                NaN
Name: 11906, dtype: object

In [27]:
organization_dupes.loc[10579, :]

address                  32-68, Oehang 1-gil,Gunsan-si
canonical_name                                 shaehan
city                                         Gunsan-si
country                      South Korea (Republic of)
country_code                                        KR
id                42e2a9a9-21ed-43bc-93cc-bb5720628d53
name                                           SHAEHAN
source            9b50d4d5-6c83-4ae8-8c47-5f722e35a78b
state                                              NaN
type              905e0790-81a3-4396-acb6-e715ce483f6a
url                                                NaN
Name: 10579, dtype: object

In [28]:
organization_dupes.loc[3, :]

address           24 Anns Nagar, KPN Colony, Tirupur-641601
canonical_name                          ahillapparelexports
city                                                    NaN
country                                               India
country_code                                             IN
id                     87bb8b6e-daab-4195-9042-19ca17b96555
name                          Ahill Apparel Exports PVT LTD
source                 48aca846-b7a8-4ecb-bf24-685395f1592a
state                                                   NaN
type                   905e0790-81a3-4396-acb6-e715ce483f6a
url                                                     NaN
Name: 3, dtype: object

In [29]:
organization_dupes.loc[1, :]

address           24 Anns Nagar, KPN Colony, Tirupur-641601
canonical_name                          ahillapparelexports
city                                                    NaN
country                                               India
country_code                                             IN
id                     87bb8b6e-daab-4195-9042-19ca17b96555
name                          Ahill Apparel Exports PVT LTD
source                 48aca846-b7a8-4ecb-bf24-685395f1592a
state                                                   NaN
type                   905e0790-81a3-4396-acb6-e715ce483f6a
url                                                     NaN
Name: 1, dtype: object